<a href="https://colab.research.google.com/github/tsolomon89/ipyd_tests/blob/main/DQF_Verifiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Intro

In [ ]:
# Cell 1: Introduction & Theory

'Self-Dual Holographic Chain & Discrete Quantum Focusing
=======================================================

**Motivation & Outline**

1. **τ-normalization & self-dual radius**
   Use τ = 2π units; at
   $$r_\star = \frac1\tau = \frac1{2\pi},$$
   the 1-D chain spacing
   $$\Delta\ell = \frac1{\pi N}$$
   makes circumference, diameter, and sphere-surface all collapse to \(1/\pi\).

2. **Static vacuum test**
   Verify the Bekenstein bound \(S \le A/4\) for the ground state via Toeplitz-FFT.

3. **Dynamic quench + local QNEC**
   Evolve a Gaussian quench and check
   \(S(t)\le A/4\) and
   \(T_{kk}-\tfrac1{2\pi}S''\ge0\).

4. **Thermal suite**
   Automate a β-scan (β ∈ [∞→0.2]) to check Bekenstein, QNEC, monotonicity, and ≥95% saturation.

5. **ML-based fit**
   Fit the saturation curve
   $$\frac{S_{\max}(\beta)}{A/4} = 1 - e^{-c\,\beta^{-p}}$$
   to extract a universal exponent \(p\).


⸻

SyntaxError: invalid syntax (<ipython-input-5-97a58dd51d3c>, line 3)

# Project

## Cell 2

In [ ]:
# Cell 2: Environment Setup (Bootstrap)

!nvidia-smi -L

import re, subprocess, sys

# Detect CUDA version
cuda_out = subprocess.check_output(["nvidia-smi", "-q"])
m = re.search(rb"CUDA Version\s*:\s*(\d+)\.(\d+)", cuda_out)
maj, minr = map(int, m.groups()) if m else (12, 2)
tag_exact   = f"{maj}{minr}"
tag_generic = f"{maj}x"

# Install NumPy, SciPy, tqdm
!{sys.executable} -m pip install -q numpy==2.0.2 scipy==1.11.4 tqdm

# Install CuPy wheel: exact first, then generic, else meta-package
def install_cupy(tag):
    try:
        # Added -v to see verbose output if this step fails
        subprocess.check_call([sys.executable, "-m", "pip", "install",
                               f"cupy-cuda{tag}==13.*", "-q"])
        return True
    except subprocess.CalledProcessError:
        return False

print("Installing CuPy for CUDA", tag_exact)
if not install_cupy(tag_exact):
    print("Exact not found; trying", tag_generic)
    # Added -v to see verbose output if this step fails
    if not install_cupy(tag_generic):
        print("Falling back to meta-package `cupy`")
        # Removed -q flag to see installation details and errors
        subprocess.check_call([sys.executable, "-m", "pip", "install", "cupy", "-v"])

import cupy, numpy
print("CuPy   :", cupy.__version__)
print("NumPy  :", numpy.__version__)
print("GPU    :", cupy.cuda.runtime.getDeviceProperties(0)['name'])

GPU 0: Tesla T4 (UUID: GPU-9ef96ff0-256b-e2fa-bc03-15e4ec78b28d)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.4 MB/s eta 0:00:00
ERROR: Cannot install numpy==2.0.2 and scipy==1.11.4 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts
Installing CuPy for CUDA 124
Exact not found; trying 12x
CuPy   : 13.3.0
NumPy  : 2.0.2
GPU    : b'Tesla T4'


## Cell 3

In [ ]:
# Cell 3: Imports & Configuration

import math, time, sys
from tqdm.notebook import tqdm

# Choose backend: CuPy (GPU) if available, else NumPy (CPU)
try:
    import cupy as xp
    from cupy.fft import fftfreq, fft, ifft
    xp.fft.config.set_plan_cache_size(16)
    backend = "CuPy GPU"
except ImportError:
    import numpy as xp
    from numpy.fft import fftfreq, fft, ifft
    backend = "NumPy CPU"

print("Backend:", backend)

# Physical & numerical constants
A4_BOUND = 1/(4*math.pi)
EPS      = 1e-12

# Notebook-wide configuration
CFG = dict(
    dtype        = "float64",
    N_vacuum     = 512,    m_vacuum = 0.15,
    dt_factor    = 0.25,
    quench_amp   = 0.8,    quench_w  = 0.04,
    steps_quench = 1500,
    beta_grid    = [math.inf,10,5,2,1,0.5,0.2],
    N_thermal    = 1024,   m_thermal = 0.05,
    steps_therm  = 1000,
    slack_tol    = 5e-6,
    mono_eps     = 1e-6,
    sat_target   = 0.95,
    WIN_LEN_PHYS = 4.0,
)

DTYPE = xp.float32 if CFG["dtype"]=="float32" else xp.float64


Backend: CuPy GPU


<ipython-input-2-b656ba728e2e>:10: FutureWarning: cupy.fft.cache.set_plan_cache_size is experimental. The interface can change in the future.
  xp.fft.config.set_plan_cache_size(16)


## Cell 4

In [ ]:
# Cell 4: Static Vacuum Test (Bekenstein)

def eigvals(M):
    if hasattr(xp.linalg, "eigvals"): return xp.linalg.eigvals(M)
    if hasattr(xp.linalg, "eig"):     return xp.linalg.eig(M)[0]
    import numpy as _np
    return xp.asarray(_np.linalg.eigvals(xp.asnumpy(M)))

def ground_cov(N, m, a):
    k = xp.asarray(2*math.pi*fftfreq(N, d=a), dtype=DTYPE)
    ω = xp.where(k==0, EPS,
                 xp.sqrt(m*m + (2*xp.sin(k*a/2)/a)**2, dtype=DTYPE))
    return ifft(0.5/ω).real, ifft(0.5*ω).real

def entropy_block(cφ, cπ, L):
    idx = xp.arange(L)
    Cφ  = cφ[(idx[:,None]-idx[None,:]) % cφ.size]
    Cπ  = cπ[(idx[:,None]-idx[None,:]) % cπ.size]
    ν   = xp.sqrt(xp.clip(eigvals(Cφ@Cπ).real,1e-18,None))
    return float(xp.sum((ν+0.5)*xp.log(ν+0.5)-(ν-0.5)*xp.log(ν-0.5)))

# Run static test
N, m = CFG["N_vacuum"], CFG["m_vacuum"]
a     = 1/(math.pi*N)
L     = N//2
cφ, cπ = ground_cov(N, m, a)
S0     = entropy_block(cφ, cπ, L)
print(f"Vacuum S = {S0:.6e}, A/4 = {A4_BOUND:.6e}")
if S0 - A4_BOUND > CFG["slack_tol"]:
    sys.exit("FAIL: Vacuum Bekenstein")
print("PASS: Vacuum Bekenstein bound ✓")


Vacuum S = nan, A/4 = 7.957747e-02
PASS: Vacuum Bekenstein bound ✓


## Cell 5

In [ ]:
# Cell 5: Dynamic Quench & Local QNEC

class Chain:
    def __init__(self,N,m):
        self.N, self.m = N, m
        self.a  = 1/(math.pi*N)
        self.dt = CFG["dt_factor"]*self.a
        self.L  = N//2

        k  = xp.asarray(2*math.pi*fftfreq(N, d=self.a), dtype=DTYPE)
        ω  = xp.where(k==0, EPS,
                      xp.sqrt(m*m+(2*xp.sin(k*self.a/2)/self.a)**2, dtype=DTYPE))
        self.ω = ω

        self.Cφ_k  = 0.5/ω
        self.Cπ_k  = 0.5*ω
        self.Cφπ_k = xp.zeros_like(ω)

        Φ0 = xp.zeros(N, dtype=DTYPE)
        Π0 = CFG["quench_amp"] * xp.exp(-((xp.arange(N)-self.L)*self.a)**2/CFG["quench_w"]**2,
                                        dtype=DTYPE)
        self.Φk, self.Πk = fft(Φ0), fft(Π0)

        cos_s = xp.cos(ω*self.dt); sin_s = xp.sin(ω*self.dt)
        self.M11, self.M12 = cos_s, sin_s/ω
        self.M21, self.M22 = -ω*sin_s, cos_s

        self._toeplitz = None
        self.E0        = self.energy()

    def _update_toeplitz(self):
        if self._toeplitz is None:
            cφ = ifft(self.Cφ_k).real
            cπ = ifft(self.Cπ_k).real
            self._toeplitz = (cφ, cπ)
        return self._toeplitz

    def step(self):
        Φk_old, Πk_old = self.Φk, self.Πk
        self.Φk = self.M11*Φk_old + self.M12*Πk_old
        self.Πk = self.M21*Φk_old + self.M22*Πk_old

        Cφ, Cπ, Cχ = self.Cφ_k, self.Cπ_k, self.Cφπ_k
        M11,M12,M21,M22 = self.M11,self.M12,self.M21,self.M22
        self.Cφ_k  = M11**2*Cφ + 2*M11*M12*Cχ + M12**2*Cπ
        self.Cπ_k  = M21**2*Cφ + 2*M21*M22*Cχ + M22**2*Cπ
        self.Cφπ_k = M11*M21*Cφ + (M11*M22+M12*M21)*Cχ + M12*M22*Cπ
        self._toeplitz = None

    def entropy(self):
        cφ, cπ = self._update_toeplitz()
        return entropy_block(cφ, cπ, self.L)

    def entropy_shift(self, δ):
        cφ, cπ = self._update_toeplitz()
        return entropy_block(cφ, cπ, self.L+δ)

    def Tkk_local(self):
        cφ, cπ = self._update_toeplitz()
        Φ = ifft(self.Φk).real
        grad = (xp.roll(Φ,-1)-Φ)/self.a
        return 0.5*(cπ[0] + 2*(cφ[0]-cφ[1])/self.a**2 + self.m**2*(Φ[self.L]**2))

    def energy(self):
        Φ = ifft(self.Φk).real; Π = ifft(self.Πk).real
        grad=(xp.roll(Φ,-1)-Φ)/self.a
        return float(0.5*xp.sum(Π**2+grad**2+self.m**2*Φ**2)*self.a + 0.5*xp.sum(self.ω))

# Run dynamic test
chain = Chain(CFG["N_vacuum"], CFG["m_vacuum"])
slack_max, qnec_fail = 0.0, False

for step in tqdm(range(CFG["steps_quench"]), desc="Dynamic Quench"):
    chain.step()
    if step % 20 != 0: continue
    S0 = chain.entropy()
    slack_max = max(slack_max, S0 - A4_BOUND)
    S_p = chain.entropy_shift(+1)
    S_m = chain.entropy_shift(-1)
    Sdd = (S_p - 2*S0 + S_m)/chain.a**2
    if chain.Tkk_local() - Sdd/(2*math.pi) < -CFG["slack_tol"]:
        qnec_fail = True
        break

if slack_max > CFG["slack_tol"]:
    sys.exit("FAIL-DYN-B: Bekenstein")
if qnec_fail:
    sys.exit("FAIL-DYN-Q: QNEC")
print("PASS: Dynamic Quench & Local QNEC ✓")


Dynamic Quench:   0%|          | 0/1500 [00:00<?, ?it/s]

PASS: Dynamic Quench & Local QNEC ✓


## Cell 6

In [ ]:
# Cell 6: DQF Thermal Suite v3.5 — fix xp.ifft & cpu fallback
import math, sys, time
from tqdm.notebook import tqdm

# Backend & FFT imports
try:
    import cupy as xp
    from cupy.fft import fftfreq, fft, ifft  # note: use `ifft`, not xp.ifft
    xp.fft.config.set_plan_cache_size(16)
    backend = "CuPy GPU"
except ImportError:
    import numpy as xp
    from numpy.fft import fftfreq, fft, ifft
    backend = "NumPy CPU"

print("Backend:", backend)

# Constants
TWO_PI, A4_BOUND = 2*math.pi, 1/(4*math.pi)
EPS   = 1e-12

# Config (reuse your CFG or redefine minimal parts)
N, m = 1024, 0.05
betas     = [math.inf, 10, 5, 2, 1, 0.5, 0.2]
steps     = 1000
dt_factor = 0.25
slack_tol = 5e-6
mono_eps  = 1e-6
sat_target= 0.95
dtype     = "float64"
WIN_LEN_PHYS = 4.0

DTYPE = xp.float32 if dtype=="float32" else xp.float64

# Helper for eigenvalues
def eigvals_gpu(M):
    if hasattr(xp.linalg, "eigvals"): return xp.linalg.eigvals(M)
    if hasattr(xp.linalg, "eig"):     return xp.linalg.eig(M)[0]
    import numpy as _np; return xp.asarray(_np.linalg.eigvals(xp.asnumpy(M)))

def coth(x):
    return 1.0 / xp.tanh(x + EPS)

# Build mode arrays
def mode_arrays(N,m,a):
    k = xp.asarray(TWO_PI*fftfreq(N, d=a), dtype=DTYPE)
    ω = xp.where(k==0, EPS,
                 xp.sqrt(m*m + (2*xp.sin(k*a/2)/a)**2, dtype=DTYPE))
    return k, ω

# Build Toeplitz first columns
def toeplitz_cols(Ck):
    return ifft(Ck).real

# Entropy from 2L×2L block without xp.block
def entropy_gaussian(Cφ, Cπ, Cχ):
    top = xp.concatenate((Cφ, Cχ), axis=1)
    bot = xp.concatenate((Cχ.T, Cπ), axis=1)
    Σ   = xp.concatenate((top, bot), axis=0)
    Σ  += xp.eye(Σ.shape[0], dtype=DTYPE)*1e-18
    ν   = xp.sqrt(xp.clip(eigvals_gpu(1j*Σ).real, 1e-18, None))
    return float(xp.sum((ν+0.5)*xp.log(ν+0.5) - (ν-0.5)*xp.log(ν-0.5)))

class ThermalChain:
    def __init__(self, N, m, beta):
        self.N, self.m = N, m
        self.a = 1/(math.pi*N)
        self.dt = dt_factor * self.a
        _, self.ω = mode_arrays(N, m, self.a)

        Cφ_k = 0.5 * coth(beta*self.ω/2) / self.ω
        Cπ_k = 0.5 * coth(beta*self.ω/2) * self.ω
        Cχ_k = xp.zeros_like(Cφ_k)
        self.Sig = xp.stack([Cφ_k, Cχ_k, Cχ_k, Cπ_k]).reshape(2,2,N)

        self.Φk = self.Πk = xp.zeros(N, dtype=DTYPE)
        cos,sin = xp.cos(self.ω*self.dt), xp.sin(self.ω*self.dt)
        self.M11, self.M12 = cos, sin/self.ω
        self.M21, self.M22 = -self.ω*sin, cos

        self._cache = None
        self.win    = min(self.N, int(round(WIN_LEN_PHYS/self.a)) + 1)
        self.win   += self.win % 2
        self.mid   = self.N//2

    def step(self):
        Φk_old, Πk_old = self.Φk, self.Πk
        self.Φk = self.M11*Φk_old + self.M12*Πk_old
        self.Πk = self.M21*Φk_old + self.M22*Πk_old

        Cφ, Cπ, Cχ = self.Sig[0,0], self.Sig[1,1], self.Sig[0,1]
        self.Sig[0,0] = self.M11**2*Cφ + 2*self.M11*self.M12*Cχ + self.M12**2*Cπ
        self.Sig[1,1] = self.M21**2*Cφ + 2*self.M21*self.M22*Cχ + self.M22**2*Cπ
        self.Sig[0,1] = self.Sig[1,0] = (
            self.M11*self.M21*Cφ + (self.M11*self.M22 + self.M12*self.M21)*Cχ +
            self.M12*self.M22*Cπ
        )
        self._cache = None

    def _cols(self):
        if self._cache is None:
            self._cache = tuple(toeplitz_cols(arr) for arr in
                                (self.Sig[0,0], self.Sig[1,1], self.Sig[0,1]))
        return self._cache

    def entropy_window(self, centre):
        half = self.win // 2
        idx = (xp.arange(centre-half, centre+half) % self.N).astype(int)
        cφ, cπ, cχ = self._cols()
        Cφ = cφ[(idx[:,None] - idx[None,:]) % self.N]
        Cπ = cπ[(idx[:,None] - idx[None,:]) % self.N]
        Cχ = cχ[(idx[:,None] - idx[None,:]) % self.N]
        return entropy_gaussian(Cφ, Cπ, Cχ)

    def Tkk_local(self, i):
        cφ, cπ, _ = self._cols()
        Φ = ifft(self.Φk).real
        grad = (xp.roll(Φ,-1) - Φ)/self.a
        return 0.5 * float(
            cπ[0] +
            2*(cφ[0]-cφ[1]) / self.a**2 +
            grad[i]**2 +
            self.m**2 * (Φ[i]**2)
        )

# Run Cell 6
ratios=[]
for β in betas:
    ch = ThermalChain(N, m, β)
    Smax=0.0
    for step in tqdm(range(steps), desc=f"β={'∞' if math.isinf(β) else β}", leave=False):
        ch.step()
        if step % 20: continue
        S = ch.entropy_window(ch.mid)
        Smax = max(Smax, S)
        if S - A4_BOUND > slack_tol: sys.exit("FAIL-B")
        Sdd = (ch.entropy_window(ch.mid+2) - 2*S + ch.entropy_window(ch.mid-2)) / (2*ch.a)**2
        if ch.Tkk_local(ch.mid) - Sdd/(2*math.pi) < -slack_tol:
            sys.exit("FAIL-Q")
    ratios.append(Smax/A4_BOUND)

if any(b+mono_eps< a for a,b in zip(ratios[:-1], ratios[1:])): sys.exit("FAIL-M")
if ratios[-1] < sat_target: sys.exit("FAIL-S")

print("DQF Thermal Suite PASSED ✓")


Backend: CuPy GPU


<ipython-input-7-687f32491ec2>:9: FutureWarning: cupy.fft.cache.set_plan_cache_size is experimental. The interface can change in the future.
  xp.fft.config.set_plan_cache_size(16)


β=∞:   0%|          | 0/1000 [00:00<?, ?it/s]

β=10:   0%|          | 0/1000 [00:00<?, ?it/s]

β=5:   0%|          | 0/1000 [00:00<?, ?it/s]

...

β=1:   0%|          | 0/1000 [00:00<?, ?it/s]

β=0.5:   0%|          | 0/1000 [00:00<?, ?it/s]

β=0.2:   0%|          | 0/1000 [00:00<?, ?it/s]

SystemExit: FAIL-S

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Cell 7

In [ ]:
# Cell 7: ML-Based Saturation Curve Fitting (no ace_tools)

import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

# Load
df = pd.read_csv("dqf_thermal_curve.csv")

# Model
def model(beta, c, p):
    return 1 - np.exp(-c * beta**(-p))

results=[]
for mass in sorted(df['mass'].unique()):
    sub = df[df['mass']==mass].copy()
    finite = sub['beta'].replace(np.inf, np.nan).dropna()
    maxb   = finite.max()
    sub['beta_fit'] = sub['beta'].replace(np.inf, maxb*1.1)
    betas  = sub['beta_fit'].values
    ratios = sub['ratio'].values

    popt, pcov = curve_fit(model, betas, ratios, p0=[1,1], maxfev=10000)
    perr       = np.sqrt(np.diag(pcov))
    chi2       = np.sum((model(betas,*popt) - ratios)**2)

    results.append({
        'mass': mass,
        'c': popt[0], 'c_err': perr[0],
        'p': popt[1], 'p_err': perr[1],
        'chi2': chi2
    })

    plt.figure(figsize=(6,4))
    plt.scatter(betas, ratios, label='Data')
    bp = np.logspace(np.log10(min(betas[betas>0])), np.log10(max(betas)), 200)
    plt.plot(bp, model(bp,*popt), '--',
             label=f"c={popt[0]:.3f}±{perr[0]:.3f}, p={popt[1]:.3f}±{perr[1]:.3f}")
    plt.xscale('log'); plt.xlabel(r'$\beta$'); plt.ylabel('S/A4')
    plt.title(f"Fit for m={mass}"); plt.legend(); plt.grid(); plt.show()

res_df = pd.DataFrame(results)
from IPython.display import display
display(res_df.style.set_caption("Saturation Fit Parameters"))


FileNotFoundError: [Errno 2] No such file or directory: 'dqf_thermal_curve.csv'

## Cell 8+

In [ ]:
# Cell 8.1 · Experiment 1: Interacting Scalar (Hartree λφ⁴)
# ────────────────────────────────────────────────────────────
import numpy as np
import math, sys
from tqdm.notebook import tqdm

# Self-consistent Hartree interaction: m_eff² = m² + λ·⟨φ²⟩
class InteractingChain(ThermalChain):
    def __init__(self, N, m0, beta, lam):
        super().__init__(N, m0, beta)
        # iterate to find m_eff
        m_eff = m0
        for _ in range(5):
            # recompute ω_k with current m_eff
            ω = xp.where(self.k==0, EPS,
                         xp.sqrt(m_eff**2+(2*xp.sin(self.k*self.a/2)/self.a)**2,dtype=DTYPE))
            # compute <φ²> = cφ_k integral
            cφ_k = 0.5*coth(beta*ω/2)/ω
            cφ0  = float(ifft(cφ_k).real[0])
            m_eff = m0 + lam*cφ0
        self.m = m_eff  # override mass
        # rebuild covariances & rotation matrices
        self.__init__(N, m_eff, beta)

# Run λ-sweep
lambdas = [0.0, 0.1, 0.5, 1.0]
for lam in lambdas:
    print(f"λ = {lam}")
    for β in CFG["betas"]:
        ch = InteractingChain(CFG["N_thermal"], CFG["m_thermal"], β, lam)
        Smax=0.0
        for _ in tqdm(range(CFG["steps_therm"]), desc=f"β={β}", leave=False):
            ch.step()
            if _%20: continue
            Smax = max(Smax, ch.entropy_window(ch.mid))
            if Smax - A4_BOUND > CFG["slack_tol"]:
                sys.exit(f"FAIL-B λ={lam}")
        print(f"  β={β}: S/A4={Smax/A4_BOUND:.4f}")
print("Experiment 1 Passed ✓\n")


In [ ]:
# Cell 8.2 · Experiment 2: Multi-Flavor Chains
# ─────────────────────────────────────────────────
# f independent scalars: total S = f·S_single, bound = f·(A/4)
flavors = [1,2,4,8]
for f in flavors:
    print(f"flavors = {f}")
    for β in CFG["betas"]:
        ch = ThermalChain(CFG["N_thermal"], CFG["m_thermal"], β)
        Smax=0.0
        for _ in tqdm(range(CFG["steps_therm"]), desc=f"β={β}", leave=False):
            ch.step()
            if _%20: continue
            Smax = max(Smax, ch.entropy_window(ch.mid))
            if f*Smax - f*A4_BOUND > CFG["slack_tol"]:
                sys.exit(f"FAIL-B f={f}")
        print(f"  β={β}: f·S/A4={f*Smax/A4_BOUND:.4f}")
print("Experiment 2 Passed ✓\n")


In [ ]:
# Cell 8.3 · Experiment 3: Radius-Deformation Sweep
# ─────────────────────────────────────────────────────
deltas = [-0.2, -0.1, 0.0, 0.1, 0.2]  # r = (1+δ)/τ
for δ in deltas:
    r = (1+δ)/TAU
    a = 2*r/CFG["N_thermal"]            # cell length = diameter/N
    A  = math.pi * r*r                  # bound = A/4 = πr²/4
    bound = A/4
    print(f"δ = {δ:+.2f}, r={(1+δ):.2f}/τ → bound = {bound:.4f}")
    for β in CFG["betas"]:
        # redefine mode arrays with new m and a
        ch = ThermalChain(CFG["N_thermal"], CFG["m_thermal"], β)
        ch.a = a; ch.L = ch.N//2
        Smax=0.0
        for _ in tqdm(range(CFG["steps_therm"]), desc=f"β={β}", leave=False):
            ch.step()
            if _%20: continue
            Smax = max(Smax, ch.entropy_window(ch.mid))
            if Smax - bound > CFG["slack_tol"]:
                sys.exit(f"FAIL-B δ={δ}")
        print(f"  β={β}: S/(A/4)={Smax/bound:.4f}")
print("Experiment 3 Passed ✓\n")


In [ ]:
# Cell 8.4 · Experiment 4: Exponent p across (m,λ,f,δ)
# ─────────────────────────────────────────────────────
import pandas as pd
from scipy.optimize import curve_fit

# Collect data from experiments 1–3 into a DataFrame `df4`
# Columns: scenario, β, ratio, param1, param2, ...
# ... (assemble code here, omitted for brevity) ...

# Define model and fit p for each scenario
def model(beta,c,p): return 1-np.exp(-c*beta**(-p))

fits=[]
for scenario, group in df4.groupby('scenario'):
    betas  = group['beta'].values
    ratios = group['ratio'].values
    popt, _ = curve_fit(model, betas, ratios, p0=[1,1], maxfev=5000)
    fits.append({'scenario':scenario, 'c':popt[0], 'p':popt[1]})

# Display results
print("Universal exponent p per scenario:")
print(pd.DataFrame(fits))
